# 需要利用MEGA之类的工具生成进化关系nwk文件

In [2]:
from Bio.Align.Applications import MuscleCommandline
from io import StringIO
from Bio import AlignIO
import sys
# sys.path.append("C:\\Users\\bio")
DATADIR="C:\\Users\\jinya\\Desktop\\bio\\pa"
# print(sys.path)
# muscle_cline = MuscleCommandline(input="{DATADIR}\\al1.fasta")
# stdout, stderr = muscle_cline()
# align = AlignIO.read(StringIO(stdout), "fasta")
# print(align)

# 获取物种相关信息

In [11]:
from ete3 import Tree, TreeStyle, NodeStyle, AttrFace, faces, TextFace,RectFace,ImgFace
import jsonlines
from ete3 import NCBITaxa
import random
ncbi = NCBITaxa()
DATADIR="C:\\Users\\jinya\\Desktop\\bio\\pa"
info, fnwk, fpng = f"{DATADIR}\\pvl1.json",  f"{DATADIR}\\pvl1.nwk",  f"{DATADIR}\\pvl1.png"

def gettax(taxid, name="species"):  # genus species
    try:
        lineage = ncbi.get_lineage(taxid)
        names = ncbi.get_taxid_translator(lineage)
        ranks = ncbi.get_rank(lineage)
        for taxid in lineage:
            if ranks[taxid] == name:
                return names[taxid]
    except:
        return
    

m = {}
with jsonlines.open(info) as fp:
    for v in fp:
        taxid = ""
        for x in v["db_xref"]:
            if "taxon:" in x:
                taxid = x[6:]
        if taxid:
            genus = gettax(taxid, "genus")
            key = "_".join(v["organism"][0].strip().split())
            m[key] = genus

def color_mapping(genus):
    # 将病毒属映射到不同的颜色
    # 这里使用一个简单的随机颜色方案，你也可以使用其他更复杂的方案
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# 创建病毒属与颜色的映射关系
genus_colors = {genus: color_mapping(genus) for genus in set(m.values()) if genus}



In [9]:
from ete3 import Tree, TreeStyle, NodeStyle, AttrFace, faces, TextFace,RectFace,ImgFace
import jsonlines
from ete3 import NCBITaxa
import random
ncbi = NCBITaxa()
info, fnwk, fpng = "pvl1.json", "pvl1.nwk", "pvl1.png"

import pickle
dmap=pickle.load(open( f"{DATADIR}\\dmap.pkl","rb"))
s = set()
for k, v in dmap.items():
    s.add(v[1])

def color_mapping(genus):
    # 将病毒属映射到不同的颜色
    # 这里使用一个简单的随机颜色方案，你也可以使用其他更复杂的方案
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# 创建病毒属与颜色的映射关系
genus_colors = {genus: color_mapping(genus) for genus in s if genus}


 # 绘制进化树

In [15]:
tree = Tree(fnwk)
# 设置树形样式
ts = TreeStyle()
ts.show_leaf_name = True  # 显示叶子节点名称
# ts.show_branch_length = True  # 显示分支长度
# ts.show_branch_support = True  # 显示分支支持率
ts.mode = "c"  # 环状模式
ts.legend_position = 1
# ts.scale = 20
# ts.branch_vertical_margin = 10 
# 自定义节点样式
for node in tree.traverse():
    if node.is_leaf():
        # 设置叶子节点样式
        genus = dmap.get(node.name)[1]
#         print(node.name)
        nstyle = NodeStyle()
        nstyle["size"] = 15
        node.set_style(nstyle)
        node.img_style["bgcolor"] =genus_colors.get(genus,"white")
        # 添加宿主信息到节点
        # host = "Host A"
        # host_face = TextFace(f"Host: {host}", ftype="Arial", fsize=10, fgcolor="blue")
        # node.add_face(host_face, column=1)
#         host_info = {}
        host_image_path = "human.png"if "HPV" in node.name else  "animal.png"
        # 添加宿主图片到节点外
        host_face = ImgFace(host_image_path, width=100, height=100)
        node.add_face(host_face, column=1, position="aligned")

    else:
        # 设置内部节点样式
        nstyle = NodeStyle()
        nstyle["fgcolor"] = "black"
        nstyle["size"] = 10
        node.set_style(nstyle)

# # 添加图例
# def add_legend(tree_style):
#     for genus, color in genus_colors.items():
#         face = TextFace(f"{genus}: ", ftype="Arial", fsize=100, fgcolor="black")
#         # color_face = TextFace(" " * 5, fsize=10, bgcolor=color)
#         color_face = RectFace(400, 100, fgcolor=color,bgcolor=color)
#         # 使用一个虚拟节点将图例放置在树的顶部或底部
#         tree_style.legend.add_face(face, column=0)
#         tree_style.legend.add_face(color_face, column=1)
# 添加图例
def add_legend(tree_style):
    colors = [ e  for e in genus_colors.items()]
    for i,(genus,color) in enumerate(colors):
        column = int(i/10)
        row = i-10*column
        face = TextFace(f"{genus}: ", ftype="Arial", fsize=100, fgcolor="black")
        color_face = RectFace(400, 100, fgcolor=color,bgcolor=color)
        # 使用一个虚拟节点将图例放置在树的顶部或底部
        tree_style.legend.add_face(face, column=column*2)
        tree_style.legend.add_face(color_face, column=column*2+1)


add_legend(ts)

# 绘制并显示进化树
tree.show(tree_style=ts)
# tree.render(fpng, w=256, h=200, units="mm", tree_style=ts)